In [1]:
import json
import os
import re
import time

import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, LlamaForCausalLM, LlamaTokenizer, pipeline
import transformers
import huggingface_hub
from src.utils import  preprocess_sample_df,save_experiment_log

prompt_dir = os.path.join('..', 'prompts')  
data_dir = os.path.join('..', 'data')  
raw_data_dir = os.path.join(data_dir,'raw')  
processed_data_dir = os.path.join(data_dir,'processed')  



sample_fpath = os.path.join(processed_data_dir,'sample_w12_stratified.csv')  
prompt_fpath=os.path.join(prompt_dir,'9_prompt_with_month.txt')  
sample_df= preprocess_sample_df(sample_fpath,prompt_fpath)



/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Process ForkProcess-2:
Process ForkProcess-6:
Process ForkProcess-3:
Process ForkProcess-4:
Process ForkProcess-1:
Process ForkProcess-5:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python

# Progress Since Last Meeting
- prompt changes:
    - added month
    - added education level
    - some fields left in English were corrected.
- data changes:
    - education level used
    - statratified sampling implemented
        - gender, ostwest, education_level
    - One Pre-covid wave is chosen
-code organization 

# System Prompt

In [7]:
from pprint import pprint


for row in sample_df.formatted_prompt.iloc[:1].values:
    #print(row)
    print( row.split("<</SYS>>",1)[0],sep='/n/n')

<s>[INST] <<SYS>> Identifizieren Sie das wichtigste Problem, mit dem Deutschland im November 2019 konfrontiert ist. Geben Sie die Antwort in einem prägnanten Satz an, konzentrieren Sie sich nur auf ein einziges Thema ohne weitere Ausführungen oder Auflistung zusätzlicher Probleme. Wiederholen Sie nicht die Informationen die Ihnen gegeben wurden, und geben Sie Ihre Antwort direkt und ohne einleitende Phrasen. Antworten Sie auf Deutsch und ausschließlich auf Deutsch, verwenden Sie keine Englische Sprache. Antworten Sie aus der Sicht eines Befragten mit den im nachfolgenden spezifizierten Eigenschaften


# Updated Prompt

In [8]:
from pprint import pprint


for row in sample_df.formatted_prompt.iloc[:15].values:
    #print(row)
    print( row.split("<</SYS>>",1)[1].strip(r'[/INST]')[:-9])

Die Befragte ist 48 Jahre alt, weiblich, mit einem Hauptschulabschluss, die in Westdeutschland lebt und hauptsächlich keine Partei unterstützt.
Die Befragte ist 64 Jahre alt, weiblich, mit einem Hauptschulabschluss, die in Westdeutschland lebt und hauptsächlich Die Linke unterstützt.
Der Befragte ist 29 Jahre alt, männlich, mit einem Realschulabschluss, der in Westdeutschland lebt und hauptsächlich eine Kleinpartei unterstützt.
Die Befragte ist 57 Jahre alt, weiblich, mit Abitur, die in Westdeutschland lebt und hauptsächlich die CDU/CSU unterstützt.
Der Befragte ist 64 Jahre alt, männlich, mit Abitur, der in Ostdeutschland lebt und hauptsächlich keine Partei unterstützt.
Die Befragte ist 64 Jahre alt, weiblich, mit einem Realschulabschluss, die in Westdeutschland lebt und hauptsächlich die FDP unterstützt.
Der Befragte ist 64 Jahre alt, männlich, mit Abitur, der in Ostdeutschland lebt und hauptsächlich keine Partei unterstützt.
Die Befragte ist 41 Jahre alt, weiblich, mit einem Realsch

# Complete List of prompt variables

In [ ]:

leaning_party_dict = {
    1: "die CDU/CSU",
    2: "die CDU",
    3: "die CSU",
    4: "die SPD",
    5: "die FDP",
    6: "die Grünen",
    7: "Die Linke",
    322: "die AfD", 
    801: "eine Kleinpartei",
    808: "keine Partei",
}
education_clauses_dict = {
    1: "ohne Schulabschluss",
    2: "mit einem Hauptschulabschluss",
    3: "mit einem Realschulabschluss",
    4: "mit Fachhochschulreife",
    5: "mit Abitur",
    9: "ist noch Schüler/in"
}

gender_dict = {
    1: "männlich",
    2: "weiblich",
    -95: "not participated"
}

ostwest_dict = {
    1: "Westdeutschland", 
    0: "Ostdeutschland",

}


# New Results

-  coronapandemie is still there , but just a few.
- partially english answers are approx. ~ 5 in 1000. 

-prompt resulting in Denglish answers:
    
`<s>[INST] <<SYS>> Identifizieren Sie das wichtigste Problem, mit dem Deutschland im Jahr 2019 konfrontiert ist. Geben Sie die Antwort in einem prägnanten Satz an, konzentrieren Sie sich nur auf ein einziges Thema ohne weitere Ausführungen oder Auflistung zusätzlicher Probleme. Wiederholen Sie nicht die Informationen die Ihnen gegeben wurden, und geben Sie Ihre Antwort direkt und ohne einleitende Phrasen. Antworten Sie auf Deutsch und ausschließlich auf Deutsch, verwenden Sie keine Englische Sprache. Antworten Sie aus der Sicht eines Befragten mit den im nachfolgenden spezifizierten Eigenschaften<</SYS>>Der Befragte ist 21 Jahre alt, männlich, mit Abitur, der in Ostdeutschland lebt und hauptsächlich keine Partei unterstützt. [/INST]`

Denglish answers:

- `Das wichtigste Problem, das Deutschland 2019 konfrontiert, ist laut mir die steigereingame Economic inequality and the lack of affordable housing.`
- `"Das wichtigste Problem, mit dem Deutschland konfrontiert ist, ist die wachsende Unzufriedenheit und Frustration der Bevölkerung, particularly among young people and those in Eastern Germany, due to a lack of perspective and the feeling of being left behind in terms of economic and social opportunities."`
